In [1]:
# Disable version warnings
import warnings
warnings.simplefilter('ignore', category=FutureWarning)
# Package setup
import os
import glob
import logging
import scanpy as sc

# Change to source directory
os.chdir('../')
# Import expert functions
from src.utils.constants import TRAINING_KEYS
from src.models._jedvi import JEDVI
# Import model run functions
import src.utils.io as io
from src.tune.run import train, full_run

# Setup logger
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

### I/O

In [2]:
# Set training and testing data directory
work_dir = '/home/xlv0877/proj_home/dl/data/merge/8a2a33dfa4d9a069'                 # Replace with path to your data directory
train_dir = os.path.join(work_dir, 'train')
test_dir = os.path.join(work_dir, 'test')
# Set model output directory
model_dir = os.path.join(work_dir, 'models', 'small')
os.makedirs(model_dir, exist_ok=True)
# Set training path
train_p = os.path.join(train_dir, 'shared_model_set_100.h5ad')                      # Select a training adata
# Set testing path
test_p = os.path.join(test_dir, 'jurkat.h5ad')                                      # Select a testing adata
# Set path to config
config_p = '../resources/params/runs/test/test.yaml'

### Two stage training

#### Stage 1: Pre-train on ELBO + supervised contrastive loss only

In [3]:
%reload_ext autoreload
%autoreload 2

In [20]:
# Pre-train config path
pre_train_config_p = '../resources/params/runs/two-stage/pre-train.yaml'
# Train model with loaded config file
model = train(adata_p=train_p, config_p=pre_train_config_p, out_dir=model_dir, verbose=False, minify_adata=False)
# Evaluate model for all data splits
model.evaluate()
# Test model with a unseen dataset
model.test(test_adata_p=test_p, cls_label='cls_label', incl_unseen=False, results_mode='save')

2025-10-28 12:10:16,745 - INFO - Loading config file: ../resources/params/runs/two-stage/pre-train.yaml
2025-10-28 12:10:16,768 - INFO - Run output directory: /home/xlv0877/proj_home/dl/data/merge/8a2a33dfa4d9a069/models/small/pre-train
2025-10-28 12:10:16,774 - INFO - Reading training data from: /home/xlv0877/proj_home/dl/data/merge/8a2a33dfa4d9a069/train/shared_model_set_100.h5ad
2025-10-28 12:10:20,949 - INFO - Setting up model.
2025-10-28 12:10:20,950 - INFO - Registered class embedding from adata.uns[cls_embedding].
2025-10-28 12:10:20,957 - WARNING - Specified control label neg;control is not in adata class labels, ignoring parameter.
2025-10-28 12:10:20,963 - INFO - Calculating external class similarities
2025-10-28 12:10:21,386 - INFO - Registered class embedding with shape: (100, 3072), using control: False
2025-10-28 12:10:21,693 - INFO - Registered classifier.
2025-10-28 12:10:21,698 - INFO - Running at: /home/xlv0877/proj_home/dl/data/merge/8a2a33dfa4d9a069/models/small/pre

<class 'src.models._jedvi.JEDVI'> Model with the following params: 
n_classes: 100, n_unseen_classes: 302, use_gene_emb: False, use_full_cls_emb: False
Training status: Not Trained
Model's adata is minified?: False

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/xlv0877/.conda/envs/pydl/lib/python3.11/site-packages/pytorch_lightning/core/saving.py:363: Skipping 'train_params' parameter because it is not possible to safely dump to YAML.
SLURM auto-requeueing enabled. Setting signal handlers.
/home/xlv0877/.conda/envs/pydl/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.
/home/xlv0877/.conda/envs/pydl/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


Training:   0%|          | 0/800 [00:00<?, ?it/s]


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [3]:
# Load a model
model_dir = '/gpfs/projects/b1042/GoyalLab/lschwartz/dl/data/merge/8a2a33dfa4d9a069/models/small/sweep/run_71/config/lightning_logs/version_0/model'
version_dir = os.path.dirname(model_dir)
output_dir = os.path.join(version_dir, 'test')
model = JEDVI.load(model_dir, adata=sc.read(train_p))

INFO     File                                                                                                      
         /gpfs/projects/b1042/GoyalLab/lschwartz/dl/data/merge/8a2a33dfa4d9a069/models/small/sweep/run_71/config/li
         ghtning_logs/version_0/model/model.pt already downloaded                                                  


2025-10-28 18:53:30,823 - INFO - Registered class embedding from adata.uns[cls_embedding].
2025-10-28 18:53:30,892 - WARNING - Specified control label neg;control is not in adata class labels, ignoring parameter.
2025-10-28 18:53:30,899 - INFO - Calculating external class similarities
2025-10-28 18:53:31,163 - INFO - Registered class embedding with shape: (100, 3072), using control: False
2025-10-28 18:53:31,470 - INFO - Registered classifier.


In [20]:
model.evaluate(output_dir=version_dir, results_mode=None)

2025-10-28 16:21:21,594 - INFO - Running model predictions on all data splits.
2025-10-28 16:21:21,607 - INFO - Using class embedding: torch.Size([100, 3072])
2025-10-28 16:21:38,310 - INFO - Generating reports.
2025-10-28 16:21:39,944 - INFO - Plotting evaluation results.
/projects/p32655/lschwartz/projects/perturbation_prediction/ExPert/vae/src/utils/plotting.py:212: UserWarning: KDE cannot be estimated (0 variance or perfect covariance). Pass `warn_singular=False` to disable this warning.
  sns.kdeplot(
2025-10-28 16:22:12,441 - INFO - Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
2025-10-28 16:22:12,463 - INFO - Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
2025-10-28 16:22:22,797 - INFO - Saving mo

#### Stage 2: Fine-tune pre-trained model with CLIP loss

In [ ]:
import pytorch_lightning as pl
from src.tune._statics import CONF_KEYS
# Specify fine-tuning config path
fine_tune_config_p = '../resources/params/runs/two-stage/fine-tune.yaml'
# Load fine-tune config params
fine_tune_config = io.read_config(fine_tune_config_p)
# Add logger to config
fine_tune_output_dir = os.path.join(version_dir, 'fine-tune')
fine_tune_config[CONF_KEYS.TRAIN]['logger'] = pl.loggers.TensorBoardLogger(fine_tune_output_dir)
# Get model params
model_kwargs = fine_tune_config[CONF_KEYS.MODEL]
# Create new model from pre-trained with frozen base encoder and decoder
fine_tune_model = JEDVI.from_base_model(model, freeze_modules=['decoder'], check_model_kwargs=False, **model_kwargs)
# Train fine-tune model
fine_tune_model.train(fine_tune_config)

2025-10-28 18:53:31,610 - INFO - Loading config file: ../resources/params/runs/two-stage/fine-tune.yaml
2025-10-28 18:53:31,636 - INFO - Registered class embedding from adata.uns[cls_embedding].
2025-10-28 18:53:31,642 - INFO - Adata has already been initialized with <class 'src.models._jedvi.JEDVI'>, loading model settings from adata.
2025-10-28 18:53:31,810 - INFO - Registered class embedding with shape: (100, 3072), using control: False
2025-10-28 18:53:32,116 - INFO - Registered classifier.
2025-10-28 18:53:32,118 - INFO - Registered external embedding aligner.
2025-10-28 18:53:32,177 - INFO - Successfully froze decoder parameters.
2025-10-28 18:53:32,178 - INFO - Epochs suggested: 117, training for 100 epochs.
2025-10-28 18:53:32,179 - INFO - Re-using pre-training data split.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


<class 'src.models._jedvi.JEDVI'> Model with the following params: 
n_classes: 100, n_unseen_classes: 302, use_gene_emb: False
Training status: Not Trained
Model's adata is minified?: False

You are using a CUDA device ('NVIDIA A100-PCIE-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/xlv0877/.conda/envs/pydl/lib/python3.11/site-packages/pytorch_lightning/core/saving.py:363: Skipping 'train_params' parameter because it is not possible to safely dump to YAML.
SLURM auto-requeueing enabled. Setting signal handlers.
/home/xlv0877/.conda/envs/pydl/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.
/home/xlv0877/.conda/envs/pydl/lib/p

Training:   0%|          | 0/100 [00:00<?, ?it/s]

### Full run for a single config

In [4]:
# Train model
results = full_run(
    config_p=config_p,              # Path to the .yaml training config
    train_p=train_p,                # Path to the .h5ad training adata
    test_p=test_p,                  # Path to the .h5ad testing adata
    model_dir=model_dir,            # Model output directory
    test_unseen=False               # Whether to test zero-shot test classification on perturbations
)

2025-10-22 20:25:31,816 - INFO - Loading config file: ../resources/params/runs/test/test.yaml
2025-10-22 20:25:31,837 - INFO - Run output directory: /home/xlv0877/proj_home/dl/data/merge/8a2a33dfa4d9a069/models/medium/test/test
2025-10-22 20:25:31,838 - INFO - Reading training data from: /home/xlv0877/proj_home/dl/data/merge/8a2a33dfa4d9a069/train/shared_model_set_w_ctrl.h5ad
2025-10-22 20:25:42,376 - INFO - Using perturbation direction to classify
2025-10-22 20:25:42,376 - INFO - Setting up model.
2025-10-22 20:25:42,376 - INFO - Adding class embedding from adata.uns[cls_embedding] to model
2025-10-22 20:25:42,388 - INFO - Adding empty control class embedding, will be learned by model.
2025-10-22 20:25:42,792 - INFO - Calculating class similarities
2025-10-22 20:25:44,969 - INFO - Registered class embedding with shape: (402, 3072), using control: False
2025-10-22 20:25:45,208 - INFO - Running at: /home/xlv0877/proj_home/dl/data/merge/8a2a33dfa4d9a069/models/medium/test/test
2025-10-22

<class 'src.models._jedvi.JEDVI'> Model with the following params: 
n_classes: 403, n_unseen_classes: 18209, use_gene_emb: False
ctrl_class: neg;control
use_learnable_control_emb: False, use_full_cls_emb: False
Training status: Not Trained

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/xlv0877/.conda/envs/pydl/lib/python3.11/site-packages/pytorch_lightning/core/saving.py:363: Skipping 'train_params' parameter because it is not possible to safely dump to YAML.
SLURM auto-requeueing enabled. Setting signal handlers.
/home/xlv0877/.conda/envs/pydl/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.
/home/xlv0877/.conda/envs/pydl/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


Training:   0%|          | 0/300 [00:00<?, ?it/s]

/home/xlv0877/.conda/envs/pydl/lib/python3.11/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/home/xlv0877/.conda/envs/pydl/lib/python3.11/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/home/xlv0877/.conda/envs/pydl/lib/python3.11/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/home/xlv0877/.conda/envs/pydl/lib/python3.11/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/home/xlv0877/.conda/envs/pydl/lib/python3.11/site-packages/anndata/_core/aligned_df.py:68: ImplicitModifica

ValueError: Could not interpret value `dataset` for `hue`. An entry with this name does not appear in `data`.

<Figure size 600x500 with 0 Axes>

### Model Training

In [ ]:
# Get config name
config_name = os.path.basename(config_p).replace('.yaml', '')
# Train model with loaded config file
train_output = train(adata_p=train_p, config_p=config_p, out_dir=model_dir, verbose=True)
# Ouput is a dictionary with these keys
TRAINING_KEYS

### Train multiple configs

In [5]:
# Run configs
model_dir = os.path.join(work_dir, 'models', 'medium')
config_dir = '../resources/params/runs/grid'
config_ps = glob.glob(f'{config_dir}/**/*.yaml', recursive=True)
config_ps

['../resources/params/runs/grid/cls_ratio.yaml',
 '../resources/params/runs/grid/shared_dim.yaml',
 '../resources/params/runs/grid/rl_weight.yaml',
 '../resources/params/runs/grid/n_negatives.yaml']

In [ ]:
# Train configs
for config_p in config_ps:
    run_name = os.path.basename(os.path.dirname(config_p))
    run_dir = os.path.join(model_dir, run_name)
    full_run(config_p=config_p, train_p=train_p, test_p=test_p, model_dir=run_dir, test_unseen=True)

2025-10-21 23:11:39,875 - INFO - Loading config file: ../resources/params/runs/grid/cls_ratio.yaml
2025-10-21 23:11:39,894 - INFO - Run output directory: /home/xlv0877/proj_home/dl/data/merge/8a2a33dfa4d9a069/models/medium/grid/cls_ratio
2025-10-21 23:11:39,897 - INFO - Reading training data from: /home/xlv0877/proj_home/dl/data/merge/8a2a33dfa4d9a069/train/full_model_set_w_ctrl.h5ad
2025-10-21 23:11:56,596 - INFO - Using perturbation direction to classify
2025-10-21 23:11:56,596 - INFO - Setting up model.
2025-10-21 23:11:56,596 - INFO - Adding class embedding from adata.uns[cls_embedding] to model
2025-10-21 23:11:56,614 - INFO - Adding empty control class embedding, will be learned by model.
2025-10-21 23:11:57,039 - INFO - Calculating class similarities
2025-10-21 23:11:59,002 - INFO - Registered class embedding with shape: (1347, 3072), using control: False
2025-10-21 23:11:59,260 - INFO - Running at: /home/xlv0877/proj_home/dl/data/merge/8a2a33dfa4d9a069/models/medium/grid/cls_ra

<class 'src.models._jedvi.JEDVI'> Model with the following params: 
n_classes: 1348, n_unseen_classes: 17264, use_gene_emb: False
ctrl_class: neg;control
use_learnable_control_emb: False, use_full_cls_emb: False
Training status: Not Trained

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/xlv0877/.conda/envs/pydl/lib/python3.11/site-packages/pytorch_lightning/core/saving.py:363: Skipping 'train_params' parameter because it is not possible to safely dump to YAML.
SLURM auto-requeueing enabled. Setting signal handlers.
/home/xlv0877/.conda/envs/pydl/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.
/home/xlv0877/.conda/envs/pydl/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


Training:   0%|          | 0/300 [00:00<?, ?it/s]

/home/xlv0877/.conda/envs/pydl/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:384: `ModelCheckpoint(monitor='f1_score_validation')` could not find the monitored key in the returned metrics: ['train_loss', 'train_loss_step', 'kl_weight', 'kl_weight_step', 'classification_ratio', 'classification_ratio_step', 'class_kl_temperature', 'class_kl_temperature_step', 'constrastive_loss_weight', 'constrastive_loss_weight_step', 'alignment_loss_weight', 'alignment_loss_weight_step', 'T_classifier_logit', 'T_classifier_logit_step', 'T_contrastive_z', 'T_contrastive_z_step', 'T_clip_z', 'T_clip_z_step', 'train_loss_epoch', 'kl_weight_epoch', 'classification_ratio_epoch', 'class_kl_temperature_epoch', 'constrastive_loss_weight_epoch', 'alignment_loss_weight_epoch', 'T_classifier_logit_epoch', 'T_contrastive_z_epoch', 'T_clip_z_epoch', 'elbo_train', 'reconstruction_loss_train', 'kl_local_train', 'kl_global_train', 'contrastive_loss_train', 'clip_z2c_train', 'clip_c2z_tra

...

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/xlv0877/.conda/envs/pydl/lib/python3.11/site-packages/pytorch_lightning/core/saving.py:363: Skipping 'train_params' parameter because it is not possible to safely dump to YAML.
SLURM auto-requeueing enabled. Setting signal handlers.
/home/xlv0877/.conda/envs/pydl/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.
/home/xlv0877/.conda/envs/pydl/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


Training:   0%|          | 0/300 [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=300` reached.
2025-10-22 03:36:10,622 - INFO - Processing train set
2025-10-22 03:36:24,092 - INFO - Using class embedding: torch.Size([1347, 3072])
2025-10-22 03:37:15,027 - INFO - Using class embedding: torch.Size([1347, 3072])
2025-10-22 03:37:34,643 - INFO - Calculating latent neighbors
2025-10-22 03:38:39,924 - INFO - Calculating latent umap
2025-10-22 03:45:52,651 - INFO - Plotting train for label: perturbation_direction
... storing 'cls_prediction' as categorical
/home/xlv0877/.conda/envs/pydl/lib/python3.11/site-packages/sklearn/metrics/_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
2025-10-22 03:46:03,099 - INFO - Plotting train for label: perturbation
2025-10-22 03:47:57,642 - INFO - Plotting train for label: dataset
100%|██████████| 10/10 [00:02<00:00,  4.81it/s]
2025-10-22 03:49:

INFO     File                                                                                                      
         /home/xlv0877/proj_home/dl/data/merge/8a2a33dfa4d9a069/models/medium/grid/shared_dim/lightning_logs/versio
         n_0/checkpoints/epoch=270-step=92897-f1_score_validation=0.04921649768948555/model.pt already downloaded  


2025-10-22 04:17:30,209 - INFO - Adding class embedding from adata.uns[cls_embedding] to model
2025-10-22 04:17:30,536 - INFO - Adding empty control class embedding, will be learned by model.
2025-10-22 04:17:31,048 - INFO - Calculating class similarities
2025-10-22 04:17:32,999 - INFO - Registered class embedding with shape: (1347, 3072), using control: False
2025-10-22 04:17:33,306 - INFO - Testing model with: /home/xlv0877/proj_home/dl/data/merge/8a2a33dfa4d9a069/test/jurkat.h5ad
2025-10-22 04:17:38,234 - INFO - Found 1348 trained class embeddings in model.
2025-10-22 04:17:38,234 - INFO - Found 18611 total class embeddings in model.
2025-10-22 04:17:38,235 - INFO - Found 1139 test perturbation(s).
2025-10-22 04:17:38,235 - INFO - Found 1031 shared perturbations between training and testing.
2025-10-22 04:17:38,235 - INFO - Found 108 unseen perturbations between training and testing.
2025-10-22 04:17:38,235 - INFO - Found 1135 perturbations in model's class embedding.


#### Manual model testing

In [ ]:
# Manual model testing

# Load best model
version_dir = 'model_dir/test/lightning_logs/version_18'
model = JEDVI.load_checkpoint(
    version_dir,
    adata=sc.read(train_p)
)
# Manually test model with control neighbor filtering
results = test(
    model, 
    test_adata_p=test_p, 
    output_dir=version_dir, 
    incl_unseen=False, 
    plot=True, 
    return_results=False, 
    min_ms=0.0,
    control_neighbor_threshold=0.1
)